# first model
## simle train and test data only  

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

import time
import gc
from tqdm import tqdm

In [2]:
!ls ../input

HomeCredit_columns_description.csv  bureau_balance.csv
POS_CASH_balance.csv		    credit_card_balance.csv
application_test.csv		    installments_payments.csv
application_train.csv		    previous_application.csv
bureau.csv			    sample_submission.csv


In [3]:
pd.options.display.max_columns = None

In [4]:
st = time.time()
train_df = pd.read_csv('../input/application_train.csv')
test_df = pd.read_csv('../input/application_test.csv')
time.time() - st

7.446819305419922

In [5]:
all_df = pd.concat([train_df, test_df])
n_train = len(train_df)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_feats = [f for f in all_df.columns if all_df[f].dtype == 'object']
for col in tqdm(categorical_feats):
    all_df[col] = all_df[col].astype('str')
    le.fit(all_df[col])
    all_df[col] = le.transform(all_df[col])

train_df = all_df[:n_train]
test_df = all_df[n_train:].drop('TARGET', axis=1)

del all_df
gc.collect()

100%|██████████| 16/16 [00:14<00:00,  1.11it/s]


14

In [6]:
train_y = train_df['TARGET'].copy()
train_X = train_df.drop('TARGET', axis = 1)

clf = LGBMClassifier(
    n_estimators=4000,
    learning_rate=0.03,
    num_leaves=30,
    colsample_bytree=.8,
    subsample=.9,
    max_depth=7,
    reg_alpha=.1,
    reg_lambda=.1,
    min_split_gain=.01,
    min_child_weight=2,
    silent=-1,
    verbose=-1,
)

clf.fit(train_X, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.03, max_depth=7, min_child_samples=20,
        min_child_weight=2, min_split_gain=0.01, n_estimators=4000,
        n_jobs=-1, num_leaves=30, objective=None, random_state=None,
        reg_alpha=0.1, reg_lambda=0.1, silent=-1, subsample=0.9,
        subsample_for_bin=200000, subsample_freq=1, verbose=-1)

In [7]:
sub_preds = clf.predict_proba(test_df)

In [8]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['TARGET'] = sub_preds

sub.to_csv('./simple_sub.csv', index=None)

In [9]:
sub.head(30)

,SK_ID_CURR,TARGET
0,100001,0.988376
1,100005,0.932246
2,100013,0.993641
3,100028,0.974833
4,100038,0.893984
5,100042,0.987169
6,100057,0.983946
7,100065,0.978521
8,100066,0.997036
9,100067,0.952499
